# **DataCamp Mock Test: Coffee Shops**

The following is a test from DataCamp as part of the Data Science Certification. 

Before starting the test I will load all required data sets provided by DataCamp. Afterwards I will perform a series of tasks. 

*The tasks are as follows:* 

1. Cleaning the data. I will work on each of the columns to create the expected dataset eitehr by replacing mising values or encoding information into the expected format. Name the cleaned file `clean_data`
1. Producing a table to show the differnce in the median number of reviews
1. Fiting a baseline using the `train.csv`
    1. Using 'validation.csv' to predict new values for `reviews` column
1. Fit a comparison model to predict the number of store review using `train.csv`
    1. Create `compare_result` dataframe with `reviews` column 

In [1]:
# import all needed libraries
import pandas as pd
import numpy as np

# import datasets saved from DataCamp
coffee = pd.read_csv('/Users/karolk/Python_Work/DataCamp/Datasets/Coffeeshops/coffee.csv')
train = pd.read_csv('/Users/karolk/Python_Work/DataCamp/Datasets/Coffeeshops/train.csv')
validation = pd.read_csv('/Users/karolk/Python_Work/DataCamp/Datasets/Coffeeshops/validation.csv')



In [2]:
# Task 1: Cleaning the data

# create a new dataframe called 'clean_data' and copy 'coffee' dataframe
clean_data = coffee.copy()

# replace missing values in 'rating' column with 0
clean_data['Rating'] = clean_data['Rating'].fillna(0)

# replace missing values in 'Reviews' column with the overall median of the column
clean_data['Reviews'] = clean_data['Reviews'].fillna(clean_data['Reviews'].median())

# replace missing values in 'Dine in option' and 'Takeout option' with 'False'
clean_data['Dine in option'] = clean_data['Dine in option'].fillna(False)
clean_data['Takeout option'] = clean_data['Takeout option'].fillna(False)

# rename the columns to Delivert option to Delivery Option, Dine in option to Dine in Option and Takeout option to Takeout Option
clean_data = clean_data.rename(columns={'Delivery option': 'Delivery Option', 'Dine in option': 'Dine in Option', 'Takeout option': 'Takeout Option'})

clean_data



,Region,Place name,Place type,Rating,Reviews,Price,Delivery Option,Dine in Option,Takeout Option
0,C,Dim Kavu,Others,4.6,206.0,$$,False,False,False
1,C,Коферум,Cafe,5.0,24.0,$$,False,False,True
2,C,Кофейня Світ Чаю,Coffee shop,5.0,11.0,$$,False,False,True
3,C,Кофейня Starcoff,Coffee shop,4.4,331.0,$$,False,True,True
4,C,"Кофейня ""Friend Zone""",Coffee shop,5.0,12.0,$$,False,True,True
...,...,...,...,...,...,...,...,...,...
195,H,Artist,Others,4.6,963.0,$$$,False,True,True
196,H,Dream Cafe,Cafe,4.3,646.0,$$,False,True,True
197,H,РУТА КАФЕ ПП КУЛІЧЕНКО К.І.,Cafe,4.6,16.0,$$,False,True,True
198,H,Ob'yektna Kava,Coffee shop,4.7,599.0,$$,False,True,True


In [3]:
# Task 2: Producing a table to show the differnce in the median number of reviews depending on the 'Rating' column

# create a new dataframe called 'reviews_by_rating' and group by 'Rating' column with four columns 'rating', 'med_review', 'min_review', 'max_review'.
reviews_by_rating = coffee.groupby('Rating').agg({'Reviews': ['median', 'min', 'max']})

# flatten the MultiIndex columns
reviews_by_rating.columns = ['_'.join(col).strip() for col in reviews_by_rating.columns.values]

# reset the index
reviews_by_rating = reviews_by_rating.reset_index()

# adjust the 'Reviews_median', 'Reviews_min', 'Reviews_max' columns to be 2 decimal places
reviews_by_rating['Reviews_median'] = reviews_by_rating['Reviews_median'].round(1)

# rename the columns to 'Rating', 'Med_Reviews', 'Min_Reviews', 'Max_Reviews'
reviews_by_rating = reviews_by_rating.rename(columns={'Rating': 'Rating', 'Reviews_median': 'Med_Reviews', 'Reviews_min': 'Min_Reviews', 'Reviews_max': 'Max_Reviews'})

# view the new dataframe
reviews_by_rating

,Rating,Med_Reviews,Min_Reviews,Max_Reviews
0,3.9,9.5,9.0,10.0
1,4.0,804.5,170.0,1439.0
2,4.1,452.5,189.0,716.0
3,4.2,497.0,385.0,609.0
4,4.3,221.5,3.0,1656.0
5,4.4,536.0,40.0,1201.0
6,4.5,688.0,27.0,2914.0
7,4.6,693.0,11.0,2931.0
8,4.7,400.0,80.0,17937.0
9,4.8,137.5,10.0,2873.0


In [4]:
# Task 3: Fiting a baseline using the `train.csv`. The value we will predict is the number of reviews. As a baseline I will use linear regressio model using only the 'Rating' column. 

# import libraries needed for the model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer


# import libraries needed for the model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# create the X_train_1, X_test_1, y_train_1, y_test variables
X_train_1 = train['Rating']
y_train_1 = train['Reviews']
X_test_1 = validation['Rating']

# impute any missing values in 'Rating' column with the median of the column
imputer = SimpleImputer(strategy='median')
X_train_1 = imputer.fit_transform(X_train_1.values.reshape(-1, 1))
X_test_1 = imputer.transform(X_test_1.values.reshape(-1, 1))

# impute the missing values in 'Reviews' column with the median of the column
y_train_1 = imputer.fit_transform(y_train_1.values.reshape(-1, 1))

# create the model
model = LinearRegression()

# fit the model
model.fit(X_train_1, y_train_1)

# predict the values
y_pred = model.predict(X_test_1)


# create a new dataframe called 'base_result' based on the 'validation' dataframe. The DF should contain the place name, rating and predicted number of reviews
base_result = validation[['Place.name', 'Rating']].copy()
base_result['Reviews'] = y_pred
# rename 'Place.name' column to 'Place name'
base_result = base_result.rename(columns={'Place.name': 'Place name'})

# adjust the 'Reviews' column to be 2 decimal places
base_result['Reviews'] = base_result['Reviews'].round(2)

# view the new dataframe
base_result


,Place name,Rating,Reviews
0,Коферум,5.0,419.80
1,Кофейня Starcoff,4.4,846.51
2,SOVA COFFEE,4.8,562.04
3,Кава Тайм,4.9,490.92
4,Dzhi,4.5,775.39
5,Koffishka,4.1,1059.86
6,Verona,4.5,775.39
7,I love coffee,4.7,633.15
8,Дом Кофе,4.6,704.27
9,Prostir.coffee,4.7,633.15


In [5]:
# Task 4: Fit a comparison model to predict the number of store review using `train.csv`. The value we will predict is the number of reviews. As a comparison model I will use linear regressio model using the 'Rating' and 'Place.type', 'Price', 'Delivery.option'.

# create the X_train_2, X_test_2, y_train_2, y_test_2 variables
X_train_2 = train[['Rating', 'Place.type', 'Price', 'Delivery.option']]
y_train_2 = train['Reviews']
X_test_2 = validation[['Rating', 'Place.type', 'Price', 'Delivery.option']]

# impute any missing values in 'Rating' column with the median of the column
X_train_2.loc[:, ['Rating']] = imputer.fit_transform(X_train_2[['Rating']])
X_test_2.loc[:, ['Rating']] = imputer.transform(X_test_2[['Rating']])

# create encoding for 'Place.type' column using label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train_2.loc[:, 'Place.type'] = le.fit_transform(X_train_2['Place.type'])
X_test_2.loc[:, 'Place.type'] = le.transform(X_test_2['Place.type'])

# transform the 'Price' column into numeric values replacing '$' with 1 and '$$' with 2 and so on
X_train_2.loc[:, 'Price'] = X_train_2['Price'].replace({'$': 1, '$$': 2, '$$$': 3})
X_test_2.loc[:, 'Price'] = X_test_2['Price'].replace({'$': 1, '$$': 2, '$$$': 3})

# replace any missing values in 'Delivery.option' column with 'False'
X_train_2.loc[:, 'Delivery.option'] = X_train_2['Delivery.option'].fillna(False)
X_test_2.loc[:, 'Delivery.option'] = X_test_2['Delivery.option'].fillna(False)

# replace the 'Delivery.option' column with 1 for 'True' and 0 for 'False'
X_train_2.loc[:, 'Delivery.option'] = X_train_2['Delivery.option'].replace({True: 1, False: 0})
X_test_2.loc[:, 'Delivery.option'] = X_test_2['Delivery.option'].replace({True: 1, False: 0})

# impute the missing values in 'Reviews' column with the median of the column
y_train_2 = imputer.fit_transform(y_train_2.values.reshape(-1, 1))

# create the model. I will use linear regression model again
model_2 = LinearRegression()

# fit the model
model_2.fit(X_train_2, y_train_2)

# predict the values
y_pred_2 = model_2.predict(X_test_2)

# create a new dataframe called 'compare_result' based on the 'validation' dataframe. The DF should contain the place name, rating and predicted number of reviews
compare_result = validation[['Place.name', 'Rating']].copy()
compare_result['Reviews'] = y_pred_2

# rename 'Place.name' column to 'Place name'
compare_result = compare_result.rename(columns={'Place.name': 'Place name'})

# adjust the 'Reviews' column to be 2 decimal places
compare_result['Reviews'] = compare_result['Reviews'].round(2)

# view the new dataframe
compare_result

,Place name,Rating,Reviews
0,Коферум,5.0,271.08
1,Кофейня Starcoff,4.4,667.27
2,SOVA COFFEE,4.8,395.91
3,Кава Тайм,4.9,328.08
4,Dzhi,4.5,1527.79
5,Koffishka,4.1,870.79
6,Verona,4.5,1495.27
7,I love coffee,4.7,463.75
8,Дом Кофе,4.6,531.59
9,Prostir.coffee,4.7,1381.27
